# Feature extraction using VGG16

In [88]:
# import packages - basics 
import pandas as pd
import os
import numpy as np
import cv2
# tf tools
import tensorflow as tf
from tensorflow import keras
# image processsing
from tensorflow.keras.preprocessing.image import (load_img,
                                                  img_to_array,
                                                  ImageDataGenerator)
# VGG16 model
from tensorflow.keras.applications.vgg16 import (preprocess_input,
                                                 decode_predictions,
                                                 VGG16)
# layers
from tensorflow.keras.layers import (Flatten, 
                                     Dense, 
                                     Dropout, 
                                     BatchNormalization)
# generic model object
from tensorflow.keras.models import Model
# optimizers
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.optimizers import SGD
#scikit-learn
from sklearn.preprocessing import LabelBinarizer
from sklearn.metrics import classification_report
# for plotting
import matplotlib.pyplot as plt
import random
from PIL import Image

## Plotting function (from last week)

In [89]:
def plot_history(H, epochs):
    plt.style.use("seaborn-colorblind")

    plt.figure(figsize=(12,6))
    plt.subplot(1,2,1)
    plt.plot(np.arange(0, epochs), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, epochs), H.history["val_loss"], label="val_loss", linestyle=":")
    plt.title("Loss curve")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.tight_layout()
    plt.legend()

    plt.subplot(1,2,2)
    plt.plot(np.arange(0, epochs), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, epochs), H.history["val_accuracy"], label="val_acc", linestyle=":")
    plt.title("Accuracy curve")
    plt.xlabel("Epoch")
    plt.ylabel("Accuracy")
    plt.tight_layout()
    plt.legend()
    plt.show()

## Loading in the data

In [98]:

classes = sorted(os.listdir("../data/images/train"))
# Remove '.DS_Store' if present
if '.DS_Store' in classes:
    classes.remove('.DS_Store')
# Convert elements to lowercase
classes = list(map(lambda x: x.lower(), classes))

Showing a sample of images from each folder 

In [ ]:
# these are all the same so can be done the same below 
datagen = ImageDataGenerator(preprocessing_function = preprocess_input)


In [ ]:
BATCH_SIZE = 32
TARGET_SIZE = (224,224)

# Split the data into categories
train_images = datagen.flow_from_directory(
    "../data/images/train",
    classes=classes,
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42)

# Split the data into categories
val_images = datagen.flow_from_directory(
    directory="../data/images/test",
    classes=classes,
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42)




## Initialize the model

__Load VGG16 *without* the classification layers__

Here we're just loading the convolutional layers and not the final classification network, using the argument ```include_top=False```



In [ ]:
# load the model
model = VGG16()

In [ ]:
# load model without classifier layers
model = VGG16(include_top=False, 
              pooling='avg',
              input_shape=(32, 32, 3))

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False

tf.keras.backend.clear_session()
# add new classifier layers - another way of adding layers - different than previous syntax we've seen before 
flat1 = Flatten()(model.layers[-1].output) 
class1 = Dense(128, activation='relu')(flat1) 
output = Dense(len(classes), activation='softmax')(class1) # Change here for how many labels there are - changed to len of classes 

# define new model
model = Model(inputs=model.inputs, 
              outputs=output)

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01, 
    decay_steps=10000,
    decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

__Train__

In [ ]:
batch_size = 32
H = model.fit(train_images,
            batch_size=batch_size,
            validation_data=val_images,
            #steps_per_epoch=train_images.samples // batch_size,
            #validation_steps=val_images.samples // batch_size,
            epochs=10,
            verbose=1)

__Evaluate__

In [ ]:
# the number is for the number of epochs 
plot_history(H, 10)

The downward tragectory of the learning curve suggests that it have more to learn. No surprise since it was only a handful of epochs with not a lot of data. 

In [ ]:
model.evaluate(test_images)


So accuracy is 44% in the test set which is 4% less than the validation accuracy during training. 

In [ ]:
predictions = model.predict(test_images, batch_size=32)
print(classification_report(test_images.classes,
                            predictions.argmax(axis=1),
                            target_names=classes)) 


Accuracy is pretty terrible. Hoping for better with data augmentation to make dataset bigger. 

## Using data augmentation

__Reload model__

In [ ]:
del model
tf.keras.backend.clear_session()

In [ ]:
# load model without classifier layers
model = VGG16(include_top=False, 
              pooling='avg',
              weights='imagenet',
              input_shape=(32, 32, 3))

# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
    
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
bn = BatchNormalization()(flat1)            ## take the outputs (image embeddings) and normalize them 
class1 = Dense(256,                         ### only difference is adding this extra hidden layer 
               activation='relu')(bn)
class2 = Dense(128, 
               activation='relu')(class1)
output = Dense(len(classes), 
               activation='softmax')(class2)

# define new model
model = Model(inputs=model.inputs, 
              outputs=output)

# compile
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01,
    decay_steps=10000,
    decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)

model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# summarize
model.summary()

In [ ]:
model = keras.Sequential([
    # Define your layers here
    # Example:
    keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    keras.layers.MaxPooling2D((2, 2)),
    keras.layers.Flatten(),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(len(classes), activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

__Define data generator__

You can see the documentation for ImageDataGenerator [here](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)

In [ ]:
train_datagen = ImageDataGenerator(rescale = 1./ 255, rotation_range = 40, horizontal_flip = True, fill_mode = 'nearest', preprocessing_function = preprocess_input)
val_datagen = ImageDataGenerator(rescale = 1./255, rotation_range = 40, horizontal_flip = True, fill_mode = 'nearest', preprocessing_function = preprocess_input)
test_datagen = ImageDataGenerator(rescale = 1./255)

In [ ]:
train_images = train_datagen.flow_from_directory(directory = train_data, batch_size = 32, target_size = (224,224), class_mode = "categorical", shuffle = False)
val_images = val_datagen.flow_from_directory(directory = val_data, batch_size = 32, target_size = (224,224), class_mode = "categorical")
test_images = test_datagen.flow_from_directory(directory = test_data, batch_size = 32, target_size = (224,224), class_mode = "categorical")


We're choosing to generate data on the fly, rather than save it to a folder. This validation split labels some as training and some as validation which we use below when training. 

__Train__

In [ ]:
batch_size = 32
H = model.fit(train_images,
            batch_size=batch_size,
            validation_data=val_images,
            steps_per_epoch=train_images.samples // batch_size,
            validation_steps=val_images.samples // batch_size,
            epochs=10,
            verbose=1)


__Inspect__

In [ ]:
plot_history(H, 10)

Still more epochs would help but does seem a bit better?

__Evaluate__

In [ ]:
model.evaluate(val_images)

accuracy has vastly improved when evaluating the test images but what does that mean when below it has barely improved?

In [ ]:
predictions = model.predict(val_images, batch_size=32)
print(classification_report(val_images.classes,
                            predictions.argmax(axis=1),
                            target_names=classes))

Have only increased the f1/accuracy by 2% by adding a hidden layer and minor data augmentation. Mango is the serious struggle, the rest seem to be doing "okay" comparatively. 

Have increased f1/accuracy by 5% by adding the batch normalization, adding one hidden layer and some minor data augmentation. Cats are the only category that is struggling. 

In [ ]:
model = VGG16()
  # load model without classifier layers
model = VGG16(include_top=False, 
              pooling='avg',
              input_shape=(32, 32, 3))
# mark loaded layers as not trainable
for layer in model.layers:
    layer.trainable = False
# add new classifier layers
flat1 = Flatten()(model.layers[-1].output)
bn = BatchNormalization()(flat1)          
class1 = Dense(256,                         
              activation='relu')(bn)
class2 = Dense(128, 
              activation='relu')(class1)
output = Dense(len(classes), 
              activation='softmax')(class2)
# define new model
model = Model(inputs=model.inputs, 
              outputs=output)
# compile
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
      initial_learning_rate=0.01,
      decay_steps=10000,
      decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)
model.compile(optimizer=sgd,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:

classes = sorted(os.listdir("../data/images/train"))
# Remove '.DS_Store' if present
if '.DS_Store' in classes:
    classes.remove('.DS_Store')
# Convert elements to lowercase
classes = list(map(lambda x: x.lower(), classes))


In [100]:
    train_datagen = ImageDataGenerator(validation_split=0.2,
                                horizontal_flip=True, 
                                rotation_range=20,  
                                preprocessing_function = preprocess_input)
    test_datagen = ImageDataGenerator(horizontal_flip=True, 
                                rotation_range=20,  
                                preprocessing_function = preprocess_input)

In [101]:
BATCH_SIZE = 32
TARGET_SIZE = (224,224)
# Training data
train_images = train_datagen.flow_from_directory(
    "../data/images/train",
    classes=classes,
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset="training",
    seed=42)
# Validation data
val_images = train_datagen.flow_from_directory(
    directory="../data/images/train",
    classes=classes,
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    subset="validation",
    seed=42)
    # test data 
test_images = test_datagen.flow_from_directory(
    "../data/images/test",
    classes=classes,
    target_size=TARGET_SIZE,
    color_mode='rgb',
    class_mode='categorical',
    batch_size=BATCH_SIZE,
    seed=42,
)


Found 1657 images belonging to 10 classes.
Found 414 images belonging to 10 classes.
Found 922 images belonging to 10 classes.


In [ ]:
H = model.fit(train_images,
        batch_size=batch_size,
        validation_data=val_images,
        steps_per_epoch=train_images.samples // batch_size,
        validation_steps=val_images.samples // batch_size,
        epochs=5,   ########## <-- adjust number here 
        verbose=1)


In [ ]:
predictions = model.predict(test_images, batch_size=32)

In [ ]:
model.evaluate(test_images)

In [ ]:
print(classification_report(test_images.classes,
                            predictions.argmax(axis=1),
                            target_names=classes)) 

In [ ]:
model2 = VGG16()
# load model without classifier layers
model2 = VGG16(include_top=False, 
            pooling='avg',
            input_shape=(32, 32, 3))
# mark loaded layers as not trainable
for layer in model2.layers:
    layer.trainable = False
tf.keras.backend.clear_session()
# add new classifier layers 
flat1 = Flatten()(model2.layers[-1].output) 
class1 = Dense(128, activation='relu')(flat1) 
output = Dense(15, activation='softmax')(class1) 
# define new model
model2 = Model(inputs=model2.inputs, 
            outputs=output)
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=0.01, 
    decay_steps=10000,
    decay_rate=0.9)
sgd = SGD(learning_rate=lr_schedule)
model2.compile(optimizer=sgd,
            loss='categorical_crossentropy',
            metrics=['accuracy'])